# Example Discovery
> Functions to discover and extract test example functions:

In [ ]:
#| default_exp cli.example_discovery

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from dataclasses import dataclass
import inspect
from typing import Dict, List, Tuple, Any, Optional
import re

from cjm_fasthtml_tailwind.cli.utils import iterate_all_modules_with_items, list_utility_modules, discover_utility_modules

In [ ]:
#| export
@dataclass
class ExampleInfo:
    """Information about a discovered test example function."""
    name: str  # Function name (e.g., 'test_spacing_basic_examples')
    module_name: str  # Module where it was found (e.g., 'spacing')
    feature: str  # Feature being demonstrated (e.g., 'basic')
    function: Any  # The actual function object
    source: str  # Source code of the function
    docstring: str  # Docstring of the function

In [ ]:
#| export
def get_example_pattern(module_name:str):
    if "." in module_name:
        # print("Submodule detected.")
        submodule_name = module_name.split(".")[-1]
        # print(f"Submodule: {submodule_name}")
        # Pattern to match test functions: test_<submodule>_<feature>_examples
        pattern = re.compile(rf'^test_{submodule_name}_(\w+)_examples$')
    else:    
        # Pattern to match test functions: test_<module>_<feature>_examples
        pattern = re.compile(rf'^test_{module_name}_(\w+)_examples$')

    return pattern

In [ ]:
#| export
def extract_test_examples_from_module(
    module: Any,  # The module to extract test examples from
    module_name: str  # The name of the module
) -> List[ExampleInfo]:  # List of ExampleInfo objects
    """Extract all test example functions from a module."""
    examples = []
    
    for name in dir(module):
        match = get_example_pattern(module_name).match(name)
        if match:
            try:
                func = getattr(module, name)
                if callable(func):
                    feature = match.group(1)
                    source = inspect.getsource(func) if hasattr(inspect, 'getsource') else ""
                    docstring = inspect.getdoc(func) or "No description available"
                    
                    examples.append(ExampleInfo(
                        name=name,
                        module_name=module_name,
                        feature=feature,
                        function=func,
                        source=source,
                        docstring=docstring
                    ))
            except (AttributeError, OSError):
                pass  # Skip if we can't get the function or its source
    
    return sorted(examples, key=lambda x: x.name)  # Sort by function name

In [ ]:
#| export
def list_all_examples(
) -> Dict[str, List[ExampleInfo]]:  # Dictionary mapping module names to their examples
    """List all test example functions across all utility modules."""
    return iterate_all_modules_with_items(extract_test_examples_from_module)

In [ ]:
#| export
def list_module_examples(
    module_name: str  # Name of the module to inspect
) -> List[ExampleInfo]:  # List of ExampleInfo objects
    """List all test example functions in a specific utility module."""
    # Find the module
    for name, module in discover_utility_modules():
        if name == module_name:
            return extract_test_examples_from_module(module, module_name)
    
    return []  # Module not found

In [ ]:
#| export
def get_example_by_name(
    module_name: str,  # Name of the module
    feature: str  # Feature name (e.g., 'basic', 'directional')
) -> Optional[ExampleInfo]:  # ExampleInfo object or None if not found
    """Get a specific example by module name and feature."""
    examples = list_module_examples(module_name)
    
    for example in examples:
        if example.feature == feature:
            return example
    
    return None

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()